In [1]:
!pip install lief

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 25.7 MB/s eta 0:00:0000:0100:01


In [7]:
import lief
import hashlib
import sys

def hash_text_section(elf_path):
    elf = lief.parse(elf_path)
    text_section = elf.get_section(".text")

    if text_section is None:
        print("No .text section found.")
        return

    content = bytes(text_section.content)
    sha256 = hashlib.sha256(content).hexdigest()

    return sha256

hash_text_section("./verified_main.elf")


'99e0cff1a6c82f4109f0ea40df1a0396fe7e88e7ea5c204a1590e19f9a2c1049'

In [2]:

def patch_ct_hash(elf_path, output_path):
    # Parse the ELF file
    elf = lief.parse(elf_path)

    # Compute SHA-256 of .text section
    text_section = elf.get_section(".text")
    if text_section is None:
        print("No .text section found.")
        return

    text_bytes = bytes(text_section.content)
    digest = hashlib.sha256(text_bytes).digest()

    print(f"Computed SHA-256: {digest.hex()}")

    # Find CT_HASH symbol
    target_sym = None
    for sym in elf.symbols:
        if sym.name == "CT_HASH":
            target_sym = sym
            break

    # target_sym now has the address of CT_HASH

    if target_sym is None:
        print("CT_HASH symbol not found.")
        return

    # Locate the section where CT_HASH lives (target_sym.value is the address)
    data_section = None
    for section in elf.sections:
        if section.virtual_address <= target_sym.value < section.virtual_address + section.size:
            data_section = section
            break

    if data_section is None:
        print("Could not find the section containing CT_HASH.")
        return

    offset = target_sym.value - data_section.virtual_address
    print(f"Patching CT_HASH at offset {offset} in section {data_section.name}")

    # Patch the bytes in the section content
    content = bytearray(data_section.content)
    content[offset:offset + len(digest)] = digest
    data_section.content = list(content)

    # Write the modified ELF
    elf.write(output_path)
    print(f"Patched ELF written to {output_path}")


patch_ct_hash("verified_main.elf", "verified_main_patched.elf")


Computed SHA-256: ccee87d1e8fcc73d158476a543334c2804dd299585d9f06ec9ab8e78e25f4b46
Patching CT_HASH at offset 8 in section .data
Patched ELF written to verified_main_patched.elf


In [3]:
import sys
import lief

def flip_bit(byte, bit_position):
    """Flip the bit at bit_position (0-7) in a byte."""
    return byte ^ (1 << bit_position)

def vma_to_file_offset(elf, vma):
    """Convert a VMA to a file offset using LIEF."""
    for segment in elf.segments:
        seg_start = segment.virtual_address
        seg_end = seg_start + segment.virtual_size
        if seg_start <= vma < seg_end:
            offset_in_segment = vma - seg_start
            return segment.file_offset + offset_in_segment
    raise ValueError(f"VMA {hex(vma)} not found in any segment")

def flip_bit_in_text_section(input_path, output_path, vma, bit_position):
    elf = lief.parse(input_path)
    if elf is None:
        print(f"❌ Failed to parse ELF: {input_path}")
        sys.exit(1)

    file_offset = vma_to_file_offset(elf, vma)

    # Read raw bytes
    with open(input_path, "rb") as f:
        data = bytearray(f.read())

    if file_offset >= len(data):
        print(f"❌ File offset {hex(file_offset)} is out of range")
        sys.exit(1)

    original_byte = data[file_offset]
    data[file_offset] = flip_bit(original_byte, bit_position)

    # Save modified binary
    with open(output_path, "wb") as f:
        f.write(data)

    print(f"✅ Flipped bit {bit_position} at VMA {hex(vma)} (file offset {hex(file_offset)})")
    print(f"📂 Saved to {output_path}")

flip_bit_in_text_section("./verified_main_patched.elf", "./altered_main.elf", 0x400100, 0)


✅ Flipped bit 0 at VMA 0x400100 (file offset 0x1100)
📂 Saved to ./altered_main.elf


In [ ]:
diff = 0xa18fa3b38793fecac18a5316658b55201ac63df89530c24f118a78b18102c54a - 0xed9b456f63a5bcd8d45e46e2cec640236c8dd2513bf342cbd75e5f8a41344e01

# // print? diff in hex power of 10
print(f"Difference in hex: {hex(diff)}")
print(f"Difference in decimal: {diff}")
print(f"Difference in scientific notation: {diff:.2e}")


Difference in hex: -0x4c0ba1bbdc11be0e12d3f3cc693aeb0351c79458a6c2807cc5d3e6d8c03188b7
Difference in decimal: -34396328055888670749958160464730542940828868522551923034469065408049195354295
Difference in scientific notation: -3.44e+76
